<a href="https://colab.research.google.com/github/hyang0129/project/blob/main/IMGNETPROTOTYPE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SETUP

In [2]:
import math, re, os
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import tensorflow as tf
import tensorflow.keras.layers as L
from sklearn import metrics
from sklearn.model_selection import train_test_split, KFold
import cv2
import tensorflow_addons as tfa
import PIL
import tensorflow.keras.backend as K
from tqdm.notebook import tqdm
import tensorflow_datasets as tfds



# HYPER PARAMETERS

In [5]:
class P():
    epochs = 100 
    net_input_dim = (224, 224, 3) 
    batch_size = 256
    num_train_img = 1,281,167


100

# DATASET DEFINITION

## AUGMENTATION

In [ ]:
def augment_image(example):

    image = example['image']

    # image = random_size(image)
    image = tf.image.random_crop(image, P.net_input_dim)
    image = tf.image.random_flip_left_right(tf.image.random_flip_up_down(image))

    example['image'] = image 

    return image 


## DATASET PREPARATION

In [ ]:
def get_dataset(ds, augment = False, shuffle = False, batch = False): 

    

## DATA SAMPLES

In [4]:
ds_train, ds_valid = tfds.load(name="imagenet2012", 
                              split=["train", "validation"], 
                              data_dir="gs://kaggledata2/imgnet", 
                              builder_kwargs={"version": "5.0.0"},)



{'file_name': <tf.Tensor: shape=(), dtype=string, numpy=b'n02097047_2079.JPEG'>,
 'image': <tf.Tensor: shape=(375, 500, 3), dtype=uint8, numpy=
 array([[[205, 205, 197],
         [203, 203, 195],
         [203, 203, 195],
         ...,
         [120, 105, 102],
         [122, 107, 104],
         [122, 108, 105]],
 
        [[205, 205, 197],
         [203, 203, 195],
         [203, 203, 195],
         ...,
         [128, 114, 111],
         [127, 116, 114],
         [129, 118, 116]],
 
        [[206, 206, 198],
         [204, 204, 196],
         [203, 203, 195],
         ...,
         [129, 119, 117],
         [130, 120, 118],
         [131, 121, 119]],
 
        ...,
 
        [[ 95,  67,  46],
         [ 95,  64,  43],
         [ 91,  61,  37],
         ...,
         [ 82,  47,  25],
         [ 94,  59,  37],
         [ 92,  57,  35]],
 
        [[ 91,  60,  40],
         [ 95,  64,  43],
         [ 87,  57,  33],
         ...,
         [ 88,  53,  31],
         [ 97,  62,  40],
     